In [ ]:
import sys,os
import pandas as pd
from rdkit import Chem
import numpy as np
from ast import literal_eval
import json
import itertools
import shap

from IPython.display import display, clear_output

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from drfp import DrfpEncoder


import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox, TextArea
from matplotlib.ticker import FormatStrFormatter

from tqdm import tqdm, trange
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=6,progress_bar=True)

from scipy.stats import pearsonr

from sklearn.feature_selection import f_regression, mutual_info_regression, r_regression, SelectKBest
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.stats import pearsonr
from math import e
from sklearn.inspection import permutation_importance

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from hpsklearn import HyperoptEstimator, any_classifier, any_preprocessing, lightgbm_regression
from hyperopt import fmin, tpe, hp

from feature_engine.selection import DropCorrelatedFeatures

from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor



In [ ]:
def get_canon(smiles: str=None)->str:
    try:
        smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smiles))
    except:
        pass
    return smiles

def write_json(in_dict: dict, filename: str):
    '''
    Write a dictionary to json file
    '''

    json_obj = json.dumps(in_dict, indent=4, cls=NpEncoder, default=set_default)
    with open(filename, 'w') as jo:
        jo.write(json_obj)

def read_json(filename: str) -> dict:
    '''
    Read in a json file and return dictionary
    '''
    with open(filename, 'r') as jo:
        json_obj = json.load(jo)
    
    return json_obj

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

def set_default(obj):
    if isinstance(obj, set):
        return list(obj)
    raise TypeError


def get_drfp(rxsmi: str=None,
            rad=3,
            nbits=2048,) -> DataStructs.cDataStructs.ExplicitBitVect:
    fps = DrfpEncoder.encode(rxsmi, n_folded_length=nbits,radius=rad)
    bv = DataStructs.ExplicitBitVect(len(fps[0]))
    bv.SetBitsFromList(np.where(fps[0])[0].tolist()) # Get index of ON bit and set it to BitVect
    # return np.asarray(bv, dtype=np.float64)
    return bv.ToList()

def expand_list_column(df: pd.DataFrame, col_name: str=None) -> tuple[pd.DataFrame,list[str]]:
    '''
    Break n-length "bitvector" column in to n columns
    '''
    col_values = df[col_name].values.tolist()
    new_col_names = [f"{col_name}_{i}" for i in range(len(col_values[0]))]
    new_df = pd.DataFrame(col_values, columns=new_col_names)
    #new_df = pd.DataFrame(np.random.randint(0,1,size=(len(new_df), len(new_col_names))), columns=new_col_names)
    new_df = pd.concat([df, new_df], axis=1)
    new_df.drop(col_name, axis=1, inplace=True)

    return new_df, new_col_names

def one_hot_encode(df, columns_to_encode):

    df_encoded = df.copy()
    ohe_columns = []
    for column in columns_to_encode:
        # One-hot encode the column using pandas get_dummies function
        one_hot_encoded = pd.get_dummies(df[column], prefix=column)
        # Add the one-hot encoded columns to the new DataFrame
        df_encoded = pd.concat([df_encoded, one_hot_encoded], axis=1)
        ohe_columns.append(one_hot_encoded.columns)
    
    ohe_columns = [col for sublist in ohe_columns for col in sublist]
    df_encoded.drop(columns=columns_to_encode, inplace=True)

    return df_encoded, ohe_columns

def get_molfp(smi: str=None,
            radius: int=3,
            nbits: int=2048,
            features: bool=False,
            as_list: bool=False,
            AddHs: bool=False) -> DataStructs.cDataStructs.ExplicitBitVect:
    try:
        mol = Chem.MolFromSmiles(smi)
    except:
        print(smi)
        return np.nan
    if AddHs==True:
        mol = Chem.AddHs(mol)

    if as_list:
        return AllChem.GetMorganFingerprintAsBitVect(mol,radius=radius,nBits=nbits,useFeatures=features).ToList()
    else:
        return AllChem.GetMorganFingerprintAsBitVect(mol,radius=radius,nBits=nbits,useFeatures=features)

def get_reg_metrics(y_true, y_pred):

    R2 = r2_score(y_true, y_pred)
    MAE = mean_absolute_error(y_true, y_pred)
    RMSE = mean_squared_error(y_true, y_pred, squared=False)

    return R2, MAE, RMSE

def smiles_to_fingerprints(smiles, radius:int=3, nBits:int=2048, features:bool=True, use_chi:bool=False, tolist:bool=False):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    if tolist == False:
        return AllChem.GetHashedMorganFingerprint(mol, radius, nBits=nBits, useFeatures=features, useChirality=False)
    else:
        return AllChem.GetHashedMorganFingerprint(mol, radius, nBits=nBits, useFeatures=features, useChirality=False).ToList()

In [ ]:
# feature selection
def select_features_f_reg(X_train, y_train, X_test):
    # configure to select all features
    fs = SelectKBest(score_func=f_regression, k='all')
    # learn relationship from training data
    fs.fit(X_train, y_train)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

# feature selection
def select_features_mi(X_train, y_train, X_test):
    # configure to select all features
    fs = SelectKBest(score_func=mutual_info_regression, k='all')
    # learn relationship from training data
    fs.fit(X_train, y_train)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

def select_features_rf(X_train, y_train, X_test, y_test,col_list):
    forest = RandomForestRegressor(n_jobs=-1,random_state=42)
    forest.fit(X_train,y_train)

    result = permutation_importance(
        forest, X_test, y_test, n_repeats=20, random_state=42, n_jobs=-1
    )
    forest_importances = pd.Series(result.importances_mean, index=col_list)
    return forest_importances

# fig, ax = plt.subplots()
# forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
# ax.set_title("Feature importances using permutation on full model")
# ax.set_ylabel("Mean accuracy decrease")
# fig.tight_layout()
# plt.show()

def feat_select(X,Y,type_,cutoff,name):
    print('----- '+name+' : '+type_+' ----')
    imp_feat = []
    # split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)
    # feature selection
    if type_ =='cf':
        X_train_fs, X_test_fs, fs = select_features_f_reg(X_train, y_train, X_test)
    elif type_ =='mi': 
        X_train_fs, X_test_fs, fs = select_features_mi(X_train, y_train, X_test)
    elif type_ == 'rf':
        fs = select_features_rf(X_train, y_train, X_test, y_test, X.columns.tolist())

    # plot the scores
    if type_ != 'rf':
        plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
        plt.axhline(cutoff,color='k',alpha=0.5,linestyle='--')
        plt.show()
        # what are scores for the features
        for i in range(len(fs.scores_)):
            if  fs.scores_[i] > cutoff:
                print('Feature %s: %f' % (X.columns[i], fs.scores_[i]))
                imp_feat.append(X.columns[i])
    else:
        plt.bar([i for i in range(len(fs.values))], fs.values)
        plt.axhline(cutoff,color='k',alpha=0.5,linestyle='--')
        plt.show()
        # what are scores for the features
        for i in range(len(fs.values)):
            if  fs.values[i] > cutoff:
                print('Feature %s: %f' % (X.columns[i], fs.values[i]))
                imp_feat.append(X.columns[i]) 
            
    return imp_feat

#important features compariosn

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 


def union(lst1, lst2): 
    lst3 = set(lst1 + lst2) 
    return lst3 

def print_feat(feat_X,name):
    
    print('\nImportant Feature for {0} : '.format(name))
    for i,feat in enumerate(feat_X):
          print(i+1,feat)
    print('-------------')

In [ ]:
def feat_ranking(X,Y,type_,cutoff,name):
    print('----- '+name+' : '+type_+' ----')
    imp_feat = []
    rows = []
    # split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)
    # feature selection
    if type_ =='cf':
        X_train_fs, X_test_fs, fs = select_features_f_reg(X_train, y_train, X_test)
    elif type_ =='mi': 
        X_train_fs, X_test_fs, fs = select_features_mi(X_train, y_train, X_test)
    elif type_ == 'rf':
        fs = select_features_rf(X_train, y_train, X_test, y_test, X.columns.tolist())

    # plot the scores
    if type_ != 'rf':
        plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
        plt.axhline(cutoff,color='k',alpha=0.5,linestyle='--')
        plt.show()
        # what are scores for the features
        for i in range(len(fs.scores_)):
            # if fs.scores_[i] > cutoff:
            # print('Feature %s: %f' % (X.columns[i], fs.scores_[i]))
            imp_feat.append(X.columns[i])
            rows.append([X.columns[i],fs.scores_[i]])
    else:
        plt.bar([i for i in range(len(fs.values))], fs.values)
        plt.axhline(cutoff,color='k',alpha=0.5,linestyle='--')
        plt.show()
        # what are scores for the features
        for i in range(len(fs.values)):
            # if fs.values[i] > cutoff:
            # print('Feature %s: %f' % (X.columns[i], fs.values[i]))
            imp_feat.append(X.columns[i]) 
            rows.append([X.columns[i],fs.scores_[i]])
    
    _df = pd.DataFrame(data=rows,columns=['Feature',f'Score_{type_}'])
    return _df

In [ ]:
ohe_prefs = ['Sulfonamide', 'Boronic Acid', 'Catalyst', 'Base', 'Solvent']

dft_cols = ['SO2N-dG_deprotonation_gas', 'SO2N-dipole', 'SO2N-electronegativity',
                'SO2N-electronic_spatial_extent', 'SO2N-hardness', 'SO2N-homo_energy',
                'SO2N-lumo_energy', 'SO2N-molar_mass', 'SO2N-molar_volume',
                'SO2N-number_of_atoms',
                'SO2N-min_APT_charge', 'SO2N-min_Mulliken_charge',
                'SO2N-min_NMR_anisotropy', 'SO2N-min_NMR_shift', 'SO2N-min_NPA_Rydberg',
                'SO2N-min_NPA_charge', 'SO2N-min_NPA_total', 'SO2N-min_NPA_valence',
                'SO2N-min_VBur', 'SO2N-max_APT_charge', 'SO2N-max_Mulliken_charge',
                'SO2N-max_NMR_anisotropy', 'SO2N-max_NMR_shift', 'SO2N-max_NPA_Rydberg',
                'SO2N-max_NPA_charge', 'SO2N-max_NPA_core', 'SO2N-max_NPA_total',
                'SO2N-max_NPA_valence', 'SO2N-max_VBur', 'SO2N-C_APT_charge',
                'SO2N-C_Mulliken_charge', 'SO2N-C_NMR_anisotropy', 'SO2N-C_NMR_shift',
                'SO2N-C_NPA_Rydberg', 'SO2N-C_NPA_charge', 'SO2N-C_NPA_core',
                'SO2N-C_NPA_total', 'SO2N-C_NPA_valence', 'SO2N-C_VBur',
                'SO2N-S_APT_charge', 'SO2N-S_Mulliken_charge', 'SO2N-S_NMR_anisotropy',
                'SO2N-S_NMR_shift', 'SO2N-S_NPA_Rydberg', 'SO2N-S_NPA_charge',
                'SO2N-S_NPA_core', 'SO2N-S_NPA_total', 'SO2N-S_NPA_valence',
                'SO2N-S_VBur', 'SO2N-O_APT_charge', 'SO2N-O_Mulliken_charge',
                'SO2N-O_NMR_anisotropy', 'SO2N-O_NMR_shift', 'SO2N-O_NPA_Rydberg',
                'SO2N-O_NPA_charge', 'SO2N-O_NPA_core', 'SO2N-O_NPA_total',
                'SO2N-O_NPA_valence', 'SO2N-O_VBur', 'SO2N-N_APT_charge',
                'SO2N-N_Mulliken_charge', 'SO2N-N_NMR_anisotropy', 'SO2N-N_NMR_shift',
                'SO2N-N_NPA_Rydberg', 'SO2N-N_NPA_charge', 'SO2N-N_NPA_core',
                'SO2N-N_NPA_total', 'SO2N-N_NPA_valence', 'SO2N-N_VBur',
                'SO2N-NPA_charge', 'CSO2N-NPA_charge', 'SO2N-HL-gap', 'SO2N-Omega',
                'SO2N-Mulliken_charge', 'CSO2N-Mulliken_charge']

solv_cols = ['Fully Soluble', 'Uniform', 'Soluble Class', 'Predicted dGsolv']
nmr_cols = ['average d1H shift', 'upfield shift from B(OH)2','remaining B(OH)2', 'broadening']

yield_col = ['RAW-MonoYield (%)']

In [ ]:
os.getcwd()

In [ ]:
if 'FeatureSelection' not in os.getcwd():
    os.chdir('FeatureSelection')
os.getcwd()

In [ ]:
df = pd.read_csv('../Data/ChanLam_full_dataset_with_features.csv')
df = df[df['Set']=='Train'].reset_index(drop=True).copy()

In [ ]:
df[dft_cols+yield_col].corr(method='spearman')

In [ ]:
sns.heatmap(df[dft_cols+yield_col].corr(method='pearson'),
           vmin=-1,vmax=1,
            cmap='coolwarm')

In [ ]:
tr = DropCorrelatedFeatures(variables=dft_cols, method='pearson', threshold=0.80)

In [ ]:
X = df[dft_cols]
y = df[yield_col].values.ravel()

In [ ]:
Xt = tr.fit_transform(X)

In [ ]:
list_of_corrs = tr.correlated_feature_sets_

In [ ]:
list_of_corrs

In [ ]:
mi_ranking = feat_ranking(X,y,'mi',0.2,'mi_ranking')

In [ ]:
ranked_feats = {}
for featset in list_of_corrs:
    _list = list(featset)
    _ranks = mi_ranking[mi_ranking['Feature'].isin(_list)].reset_index(drop=False).copy()
    
    _topfeat = _ranks.sort_values(by='Score_mi',ascending=False).reset_index(drop=True).loc[0,'Feature']
    _restfeats = _ranks.sort_values(by='Score_mi',ascending=False).reset_index(drop=True).loc[1:,'Feature'].values.tolist()
    ranked_feats[_topfeat] = _restfeats

In [ ]:
# drop_corrs = list(itertools.chain.from_iterable(list(ranked_feats.values())))
drop_corrs = list(itertools.chain.from_iterable(list(ranked_feats.values())))
print(len(drop_corrs))
# drop_corrs

In [ ]:
# tr.correlated_feature_dict_

In [ ]:
# tr.features_to_drop_

In [ ]:
final_cols = list(set(dft_cols+nmr_cols+solv_cols)-set(drop_corrs))

In [ ]:
final_cols

In [ ]:
feat_temp = df.copy()
scaler = StandardScaler()
# X = feat_temp[dft_cols+nmr_cols+solv_cols].copy()
X = feat_temp[final_cols]
# X = feat_temp.drop(columns=drop_cols).copy()
cols = X.columns

X[cols] = scaler.fit_transform(X[cols])

# Y = feat_temp['MonoYield (%)']
Y = feat_temp['RAW-MonoYield (%)']

In [ ]:
imp_feat_cf = feat_select(X,Y,'cf',363.939948*0.25,'RAW-MonoYield (%)') # max-val * 0.25
imp_feat_mi = feat_select(X,Y,'mi',0.216898*0.5,'RAW-MonoYield (%)')
imp_feat_rf = feat_select(X,Y,'rf',0.376505*0.25,'RAW-MonoYield (%)') # max-val * 0.25

### Assign to lists
feat_int = intersection(imp_feat_cf, imp_feat_mi)
feat_un = union(imp_feat_cf, imp_feat_mi)
feat_rf = imp_feat_rf

print_feat(feat_int,'Intersection')
print_feat(feat_un,'Union')

In [ ]:
tab = '\t'
pca = PCA(n_components=.98)
X_pca = pca.fit_transform(X)
print(f'Explained variance: {pca.explained_variance_ratio_.sum():.1%}')

# thrs = 0.35
thrs = np.max(np.abs(pca.components_)) * 0.5
auto_select = []
for i, name in enumerate(pca.feature_names_in_):
    fe = []
    for pcn in pca.components_:
        fe.append(round(pcn[i], 3))

    arrfe = np.asarray(fe, dtype=np.float64)
    if np.max(np.abs(arrfe)) > thrs:# or arrfe.max() > thrs:
        auto_select.append(name)
    # print(f'{name:<35}{fe}')

print('\n'+f'Autoselection from {pca.components_.shape[0]} PCs with {thrs = :.3f}')
print('\n'.join(auto_select))


In [ ]:
model = LGBMRegressor(n_jobs=-1,
                     learning_rate=0.01,
                     n_estimators=1024,
                     reg_lambda=0.01,
                     reg_alpha=0.01,
                     random_state=42
                     ).fit(X, Y)

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn, transformers, Spark, etc.)
explainer = shap.Explainer(model)
shap_values = explainer(X)

# visualize the first prediction's explanation
shap.plots.waterfall(shap_values[0])

In [ ]:
shap.plots.waterfall(shap_values[0])

In [ ]:
shap_sel_idxs = np.where(np.abs(shap_values[0].values)>=0.6)
X.columns[np.where(np.abs(shap_values[0].values)>=0.6)].tolist()

In [ ]:
X.columns[shap_sel_idxs].tolist()

In [ ]:
# shap_values[0][np.where(np.abs(shap_values[0].values)>=0.40)]

In [ ]:
import lightgbm
lightgbm.__version__

In [ ]:
shap.__version__

In [ ]:
plt.clf()
shap.plots.waterfall(shap_values[0][shap_sel_idxs], show=False)
# plt.savefig('SHAP_Waterfall_TrainVal_20240416.svg',transparent=True,bbox_inches='tight')
plt.savefig('SHAP_Waterfall_TrainVal_20240416.png',transparent=True,bbox_inches='tight',dpi=600)

plt.clf()

In [ ]:
plt.clf()
# shap.plots.beeswarm(shap_values)
shap.plots.beeswarm(shap_values, show=False)
# plt.savefig('SHAP_Beeswarm_TrainVal_20240416.svg',transparent=True,bbox_inches='tight')
plt.savefig('SHAP_Beeswarm_TrainVal_20240416.png',transparent=True,bbox_inches='tight',dpi=600)

plt.clf()

In [ ]:
shap_interaction_values = explainer.shap_interaction_values(X)
shap_interaction_values[0]

In [ ]:
sns.heatmap(shap_interaction_values[0],
cmap='coolwarm',vmin=-1,vmax=1)

In [ ]:
sns.heatmap(shap_interaction_values[0][shap_sel_idxs[0],:][:,shap_sel_idxs[0]],
cmap='coolwarm',vmin=-1,vmax=1,annot=True,fmt='.2f')

In [ ]:
shap_interaction_values[0].shape

In [ ]:
# shap_interaction_values[shap_sel_idxs[0],:][:,shap_sel_idxs[0]]

In [ ]:
shap_sel_idxs[0]

In [ ]:
a = np.arange(20).reshape((5,4))

In [ ]:
a

In [ ]:
a[[0,1,3], :][:, [0,2]]

In [ ]:
feature_dict = {'Only OHE': [],
                'All Properties': dft_cols+nmr_cols+solv_cols,
                'DFT': dft_cols,
                'NoCoLinear': final_cols,
                'F-regression': imp_feat_cf,
                'MutualInfo': imp_feat_mi,
                'F-MI_Intersection': feat_int,
                'F-MI_Union': list(feat_un),
                'RandomForest': feat_rf,
                'PCA': auto_select,
                'SHAP': X.columns[shap_sel_idxs].tolist(),
                }

In [ ]:
write_json(feature_dict, 'Feature_selection.json')